In [ ]:
# load package to import excel files
library(readxl)

# import first dataset “cyclistic_2019
cyclistic_2019 <- read_excel("/kaggle/input/cyclistic-data-gcc-exercise/cyclistic_2019.xlsx")

# import first dataset “cyclistic_2020
cyclistic_2020 <- read_excel("/kaggle/input/cyclistic-data-gcc-exercise/cyclistic_2020.xlsx")

# Introduction

This work is part of the final assignment for the Google Career Certificate for Data Analyst course on the Coursera platform. Data and context were provided by Google Career Certificate. How to process them is a personal assignment.  

The assignment consists in analyzing data, for the bike-share fictional company "Cyclistic” in Chicago, as a junior data analyst in the company's marketing team.  

Initial situation : the marketing director wants to change the marketing strategy. She believes that the future success of the company depends on the number of annual subscriptions. Until now, the target audience has been the general public, but she now thinks it's a good idea to focus on occasional users and get them to sign up for an annual subscription. 

Key information :  

* 5824 bicycles that are geotracked and locked  
* 692 stations across Chicago  
* Bikes can be unlocked from one station and returned to any other station anytime
* There are two categories of travel pass: casual passes (one journey and one day) and annual season tickets 
* We have two dataframes : one for 2019 and the other for 2020  

Questions from the client :  

* How do *annual members* (customers who purchase annual memberships) and *casual riders* (customers who purchase single-ride or full-day passes) use Cyclistic bikes differently  
* Identify trends in service usage  
* Determine the reasons why *casual riders*  become *annual members*  
* How can Cyclistic use digital media to influence *casual riders* to become *annual members*

# Setting up my environment

## Imported Datasets

* cyclistic_2019 : about 2019 data  
* cyclistic_2020 : about 2020 data  

Theses dataframes has already been imported.  

## Loaded Packages

The readxl package has already been loaded to enable us to import dataframes.  

In [ ]:
# Load all the packages we will use in this analysis
library(tidyverse)
library(ggplot2)
library(openxlsx)
library(lubridate)

# Data cleaning process

At the end of each cleaning operation, a check will be performed using tools (such as sum, colnames, summary, str, ...). To simplify this document, we won't always include them in code chuncks.

## Explore dataframes

Let's explore the documents as they were originally given to us, to determine the clean-up actions to come.

In [ ]:
# Structure of cyclistic_2019
str(cyclistic_2019)

# Main statistics of cyclistic_2019
summary(cyclistic_2019)

# Structure of cyclistic_2020
str(cyclistic_2020)

# Main statistics of cyclistic_2020
summary(cyclistic_2020)

### Remarks following observation of these descriptions :  

#### About the structure :  

* the number of columns is different (12 for cyclistic_2019, 13 for cyclistic_2020)  
* none of the columns have the same name  
* some columns don't match  
* the datatype is the right one for each column in each document, but we may need to harmonize it for future merging of these dataframes  
* both documents are already in wide data format  
* both documents are already anonymized  
* comparable number of observations (365,069 for cyclistic_2019, 426,887 for cyclistic_2020)  
* in both document, there is no column with the hours only, to be able to identify peak activity times

#### About the summary (main statistics) :  

* in cyclistic_2019 some birth years are not consistent (individuals over 100 years old at the time of data collection)  
* in cyclistic_2019 maximum bike_id (6471) is greater than the number of Cyclistic bikes (5824)  

##### Time data :

* The maximum travel time in cyclistic_2019 is equivalent to around 123 days (10628400 seconds)  
* The maximum in the end-of-trip column of cyclistic_2019 and in cyclistic_2020 seems inconsistent  
* For each dataframe, the period covered by the data could be to be from January to March (it would make the data comparable)  
* In cyclistic_2020 trip duration column is missing  

##### Geographic data :

* The is no column about geographic coordinates in cyclistic_2019 (instead of 4 in cyclistic_2020 about departure and destination latitude and longitude)
* There is few missing data about geographic coordinates in cyclistic_2020  
* Both dataframes contain the names of the stations.If they are formatted in the same way (which seems to be), most of the missing geographical data could be found   
* In both dataframes the maximum in departure and destination station (665 in cyclistic_2019, 675 in cyclistic_2020)is less than the total of the number of station in Cyclistic network (692). This means that all the stations in the network were not used at all during the periods studied  

## Harmonize columns

### Add a year column

To be able to differentiate the year of the data once the data frames have been merged.  

In [ ]:
# Add year in cyclistic_2019 in numeric datatype
cyclistic_2019 <- cyclistic_2019 %>%
  mutate(year = as.numeric("2019"))

# Add year in cyclistic_2020 in numeric datatype
cyclistic_2020 <- cyclistic_2020 %>%
  mutate(year = as.numeric("2020"))

### Add trip duration column in cyclistic_2020

In [ ]:
# Create a trip_duration column in cyclistic_2020 by subtracting the departure date from the arrival date by difftime function
cyclistic_2020 <- cyclistic_2020 %>%
  mutate(trip_duration = as.numeric(difftime(ended_at, started_at, units = "secs")))

### Add empty missing columns

In [ ]:
# Add departure_latitude in cyclistic_2019 in numeric datatype
cyclistic_2019 <- cyclistic_2019 %>%
  mutate(departure_latitude = as.numeric(NA))

# Add departure_longitude in cyclistic_2019 in numeric datatype
cyclistic_2019 <- cyclistic_2019 %>%
  mutate(departure_longitude = as.numeric(NA))

# Add destination_latitude in cyclistic_2019 in numeric datatype
cyclistic_2019 <- cyclistic_2019 %>%
  mutate(destination_latitude = as.numeric(NA))

# Add destination_longitude in cyclistic_2019 in numeric datatype
cyclistic_2019 <- cyclistic_2019 %>%
  mutate(destination_longitude = as.numeric(NA))

# Add gender in cyclistic_2020 in character datatype
cyclistic_2020 <- cyclistic_2020 %>%
  mutate(gender = as.character(NA))

# Add birth_year in cyclistic_2020 in numeric datatype
cyclistic_2020 <- cyclistic_2020 %>%
  mutate(birth_year = as.numeric(NA))

## Remove extra columns

A closer look reveals rideable_type in cyclistic_2020 contains only one entry: “docked_bike”. In addition, it wasn't compatible with cyclistic_2019's bike_id column, so we decided to simply delete it.

In [ ]:
# Remove rideable_type in cyclistic_2020
cyclistic_2020$rideable_type <- NULL

We decide to delete bike_id too in cyclistic_2019. This column contains the identifier of each cyclistic bike, and as we have no other information on these bikes it will be difficult to group them by year of purchase, for example, to find out their condition. As the original data is saved, we'll be able to retrieve it should we ever need it. Without grouping them, there's not much point in identifying the most frequently used bikes.

In [ ]:
# Remove bikeid in cyclistic_2019
cyclistic_2019$bikeid <- NULL

### Harmonize column names

In [ ]:
# Rename columns in cyclistic_2019
cyclistic_2019 <- cyclistic_2019 %>%
  rename(
    departure_time = start_time,
    destination_time = end_time,
    trip_duration = tripduration,
    departure_station_id = from_station_id,
    departure_station_name = from_station_name,
    destination_station_id = to_station_id,
    destination_station_name = to_station_name,
    travel_pass = usertype,
    birth_year = birthyear )

# Rename columns in cyclistic_2020
cyclistic_2020 <- cyclistic_2020 %>%
  rename(
    trip_id = ride_id,
    departure_time = started_at,
    destination_time = ended_at,
    departure_station_id = start_station_id,
    departure_station_name = start_station_name,
    destination_station_id = end_station_id,
    destination_station_name = end_station_name,
    travel_pass = member_casual,
    departure_latitude = start_lat,
    departure_longitude = start_lng,
    destination_latitude = end_lat,
    destination_longitude = end_lng )

### Harmonize column data types

Trip ID don't have the same encoding between the two data frame, but as they are unique we can use them in the same column. However, to be able to merge data, we have to use the same datatype.

In [ ]:
# Convert trip_id in character datatype in cyclistic_2019
cyclistic_2019 <- cyclistic_2019 %>%
  mutate(trip_id = as.character(trip_id))

## Basic global cleaning

### Remove excess white spaces

In each column where the datatype is character, we will remove excess white space before, after and in the middle of the string. 

We're going to use the *trimws* function to remove spaces at the beginning and end, and the *gsub* argument to remove excess spaces in the middle of the string, where we should define what we're removing ( \\s+ : when there's a space or more) and what we're replacing it with: (” ”, an empty space), and finally what we're applying it to: ( . : to each column of the dataframe). We'll repeat this operation for each dataframe. 

In [ ]:
# Remove extra white spaces in cyclistic_2019
cyclistic_2019 <- cyclistic_2019 %>%
  mutate_if(is.character,
    ~trimws(gsub("\\s+", " ", .)))

# Remove extra white spaces in cyclistic_2020
cyclistic_2020 <- cyclistic_2020 %>%
  mutate_if(is.character,
    ~trimws(gsub("\\s+", " ", .)))

### Uniform case

We will use two case styles in this dataset: we will capitalize the beginning of each word for the station name columns (using *stringr::str_to_title*), and we will lowercase the other datatype character columns (using *tolower*). *accross* allows us to apply the function to several columns at once.

In [ ]:
# Uniform case in cyclistic_2019
cyclistic_2019 <- cyclistic_2019 %>%
  mutate(across(
    where(is.character) & !c(departure_station_name, destination_station_name),
    tolower)) %>%
  mutate(across(
    c(departure_station_name, destination_station_name),
    stringr::str_to_title))

# Uniform case in cyclistic_2020
cyclistic_2020 <- cyclistic_2020 %>%
  mutate(across(
    where(is.character) & !c(departure_station_name, destination_station_name),
    tolower)) %>%
  mutate(across(
    c(departure_station_name, destination_station_name),
    stringr::str_to_title))

### Check / remove duplicate

In [ ]:
# Check for duplicates by trip_id in cyclistic_2019
sum_dupl_19 <- sum(duplicated(cyclistic_2019$trip_id))
# Print result
if(sum_dupl_19 == 0) { 
    print("No duplicate in cyclistic_2019, no need to remove them")
  } else { 
    print(paste("Whoops, ", sum_dupl_19, "duplicates. We have to remove them !"))}

# Check for duplicates by trip_id in cyclistic_2020
sum_dupl_20 <- sum(duplicated(cyclistic_2020$trip_id))
# Print result
if(sum_dupl_20 == 0) { 
    print("No duplicate in cyclistic_2020, no need to remove them")
  } else { 
    print(paste("whoops, ", sum_dupl_20, "duplicates. We have to remove them !"))}

## Geographic data

### Station name and station ID columns

There are four columns in our dataset about station names (departure_station_name and destination_station_name in both dataframe) and also four about station ID (departure_station_id and destination_station_id in both dataframe).  

We'll compare the number of different station names in each column of each dataframe with the number of different ID. If the number is the same (without missing data), we can assume that data in these columns is error-free and ready for use.

In [ ]:
# Counts the number of unique entries, omitting missing values in each column about name or ID station of our dataset
l_u_dep_name_19 <- length(unique(na.omit(cyclistic_2019$departure_station_name)))
l_u_dep_id_19 <- length(unique(na.omit(cyclistic_2019$departure_station_id)))
l_u_des_name_19 <- length(unique(na.omit(cyclistic_2019$destination_station_name)))
l_u_des_id_19 <- length(unique(na.omit(cyclistic_2019$destination_station_id)))
l_u_dep_name_20 <- length(unique(na.omit(cyclistic_2020$departure_station_name)))
l_u_dep_id_20 <- length(unique(na.omit(cyclistic_2020$departure_station_id)))
l_u_des_name_20 <- length(unique(na.omit(cyclistic_2020$destination_station_name)))
l_u_des_id_20 <- length(unique(na.omit(cyclistic_2020$destination_station_id)))

# Length comparison between departure_station_name and departure_station_id ID in cyclistic_2019
if (l_u_dep_name_19 == l_u_dep_id_19) { 
  print("length of departure_station_name and departure_station_id is the same, data is useable (cyclistic_2019)")
} else { 
  print("Warning, there are some errors: length of departure_station_name and departure_station_id is NOT the same (cyclistic_2019)")}

# Length comparison between destination_station_name and destination_station_id in cyclistic_2019
if (l_u_des_name_19 == l_u_des_id_19) { 
  print("length of destination_station_name and destinatione_station_id is the same, data is useable (cyclistic_2019)")
} else { 
  print("Warning, there are some errors: length of destination_station_name and destination_station_id is NOT the same (cyclistic_2019)")}

# Length comparison between departure_station_name and departure_station_id in cyclistic_2020
if (l_u_dep_name_20 == l_u_dep_id_20) { 
  print("length of departure_station_name and departure_station_id is the same, data is useable (cyclistic_2020)")
} else { 
  print("Warning, there are some errors: length of departure_station_name and departure_station_id is NOT the same (cyclistic_2020)")}

# Length comparison between destination_station_name and destination_station_id in cyclistic_2020
if (l_u_dep_name_20 == l_u_dep_id_20) { 
  print("length of destination_station_name and destination_station_id is the same, data is useable (cyclistic_2020)")
} else { 
  print("Warning, there are some errors: length of destination_station_name and destination_station_id is NOT the same (cyclistic_2020)")}

### Latitude and longitude

cyclistic_2020 contains the latitude and longitude of each station. We're going to create and complete these same columns for cyclistic_2019. To do this, we first need to create a correspondence table between station names, latitude and longitude. From this table we'll be able to fill in the missing data in each dataframe.  

In [ ]:
# Create a correspondence table between station name, latitude and longitude
correspondence_table_station_name_lat_long <- cyclistic_2020 %>%
  select(station_name = departure_station_name, 
         latitude = departure_latitude, 
         longitude = departure_longitude) %>%
  bind_rows(
    cyclistic_2020 %>%
      select(station_name = destination_station_name, 
             latitude = destination_latitude, 
             longitude = destination_longitude)) %>%
  drop_na() # Remove rows with missing data

# Checks for each station_name entry, the corresponding latitude and longitude are the same
check_corresponding_lat_long <- correspondence_table_station_name_lat_long %>%
  group_by(station_name) %>%
  summarise(n_lat_long = n_distinct(latitude, longitude)) %>%
  filter(n_lat_long > 1)
# Print result
if (nrow(check_corresponding_lat_long) == 0) { print("Data is clean, you can remove duplicates.")
} else {  print("There are some stations with wrong data, we have to fix it")}

In [ ]:
# Keep only one row for each station name (and keep other colomns)
correspondence_table_station_name_lat_long <- correspondence_table_station_name_lat_long %>%
  distinct(station_name, .keep_all = TRUE)

Our correspondence table between station names, latitude and longitude is complete. Now we will use it to complete missing data in cylistic_2019 (There is just one missing data in theses columns in cyclistic_2020 but we can't fix it because none of these columns have been completed). As the stations used in 2019 and 2020 are a bit different, there will necessarily be some missing data in cyclistic_2019.  

In [ ]:
# Complete missing data about departure latitude and longitude in cyclistic_2019
# Here, left data frame is cyclistic_2019, right dataframe is correspondence_table_station_name_lat_long
# When doing a left_join, if both dataframes have columns with the same name, R automatically renames them with a .x and a .y at the end of the name
cyclistic_2019 <- cyclistic_2019 %>%
  left_join(correspondence_table_station_name_lat_long, 
            by = c("departure_station_name" = "station_name")) %>%
  mutate(
    departure_latitude = ifelse(is.na(departure_latitude), latitude, departure_latitude),
    departure_longitude = ifelse(is.na(departure_longitude), longitude, departure_longitude)) %>%
  select(-latitude, -longitude)  # Remove temporary columns

# Complete missing data about destination latitude and longitude in cyclistic_2019
cyclistic_2019 <- cyclistic_2019 %>%
  left_join(correspondence_table_station_name_lat_long, 
            by = c("destination_station_name" = "station_name")) %>%
  mutate(
    destination_latitude = ifelse(is.na(destination_latitude), latitude, destination_latitude),
    destination_longitude = ifelse(is.na(destination_longitude), longitude, destination_longitude)) %>%
  select(-latitude, -longitude)  # Remove temporary columns

## Time data

### Trip duration

As a reminder, both dataframes appear to cover the period from January to March. However, in both, we noticed the presence of some anomalous data. We're going to check the validity of our data by deleting all data longer than 24h (96400 seconds).

In [ ]:
# Delete trip duration longer than 24h (86400 seconds) in cyclistic_2019
cyclistic_2019 <- cyclistic_2019 %>%
  filter(trip_duration <= 86400)

# Discover the date of the last bike return in cyclistic_2019
max(cyclistic_2019$destination_time)

# Delete trip duration longer than 24h (86400 seconds) in cyclistic_2020
cyclistic_2020 <- cyclistic_2020 %>%
  filter(trip_duration <= 86400)

# Discover the date of the last bike return in cyclistic_2020
max(cyclistic_2020$destination_time)

In both dataframes, the return date of the last bike in our data is April 1, so the data is usable.

### Create a time column and a date column

To be able to identify days and peak times independently, we'll create new columns. 

In [ ]:
# Create a column with only hour then a column with only the day of week for departure_time in cyclistic_2019
cyclistic_2019 <- cyclistic_2019 %>%
  mutate(
    departure_hour = hour(departure_time),
    departure_day_of_week = as.character(wday(departure_time, label = TRUE, abbr = FALSE)))

# Create a column with only hour then a column with only the day of week for destination_time in cyclistic_2019
cyclistic_2019 <- cyclistic_2019 %>%
  mutate(
    destination_hour = hour(destination_time),
    destination_day_of_week = as.character(wday(destination_time, label = TRUE, abbr = FALSE)))

# Create a column with only hour then a column with only the day of week for departure_time in cyclistic_2020
cyclistic_2020 <- cyclistic_2020 %>%
  mutate(
    departure_hour = hour(departure_time),
    departure_day_of_week = as.character(wday(departure_time, label = TRUE, abbr = FALSE)))

# Create a column with only hour then a column with only the day of week for destination_time in cyclistic_2020
cyclistic_2020 <- cyclistic_2020 %>%
  mutate(
    destination_hour = hour(destination_time),
    destination_day_of_week = as.character(wday(destination_time, label = TRUE, abbr = FALSE)))

## User & Bike Data  

### Travel pass columns

For each row, this column is used to identify if the travel pass held by the cyclist is a temporary pass (one trip or one day) or if the cyclist has used an annual pass. We will use “member” to refer to the annual pass, "casual" to refer to an occasional pass.

In [ ]:
# Turns all “subscriber” into “member” and all “customer” into “casual” in cyclistic_2019
cyclistic_2019 <- cyclistic_2019 %>%
  mutate(travel_pass = case_when(
    travel_pass == "subscriber" ~ "member",
    travel_pass == "customer" ~ "casual"))

# Checks for errors in cyclistic_2019
unique(cyclistic_2019$travel_pass)

# Checks for errors in cyclistic_2020
unique(cyclistic_2020$travel_pass)

### Birth year columns

In cyclistic_2019 only, there is a birth year column. Some data are unexpected: the minimum year is 1900, which would make a user over 100 years old. 

In [ ]:
# Turn birth years in or before 1900 in NA
cyclistic_2019 <- cyclistic_2019 %>%
  mutate(birth_year = ifelse(birth_year < 1919, NA, birth_year))

## Merging dataframes

In [ ]:
# Merging cyclistic_2019 and cyclistic_2020
cyclistic_ready_for_analysis <- bind_rows(cyclistic_2019, cyclistic_2020)

#Structure of our new dataframe
str(cyclistic_ready_for_analysis)

# Main statistics of our new dataframe
summary(cyclistic_ready_for_analysis)

Now that our data is clean, we can move on to analyzing it.

# Data analysis

## Member vs casual

First, for information only, we want to know percentage of casual for each year.

In [ ]:
cyclistic_ready_for_analysis %>%
  group_by(year) %>%
  summarise(
    sum_casual = sum(travel_pass == "casual"),
    sum_member = sum(travel_pass == "member"),
    percentage_casual = round(sum_casual / n() * 100, 2))

We are seeing an increase in the use of cyclistic, both casual and annual pass. This means that there are probably some riders “casuals” who have been converted to “members” between 2019 and 2020.

We want to better know members and casuals users profiles.

## Gender

We know that gender data is only available for 2019, so we'll only be looking at this data.

In [ ]:
cyclistic_ready_for_analysis %>%
  
# Filter data to see 2019 only then group all "member" and group all “member” values 
# and group all “casual” values
  filter(year == 2019) %>%
  group_by(travel_pass) %>%
  
  summarise(
    sum_male = sum(gender == "male", na.rm = TRUE),
    sum_female = sum(gender == "female", na.rm = TRUE),
    sum_na = sum(is.na(gender))) %>%
  
# Calculate the percentages: of women to men, of men to women (excluding missing data). 
# Then calculate the percentage of missing data in relation to the gendered data recorded.
  mutate(
    percentage_female_no_na = round((sum_female / (sum_male + sum_female)) * 100, 2)) %>%
   mutate(
    percentage_male_no_na = round((sum_male / (sum_male + sum_female)) * 100, 2)) %>%
   mutate(
    percentage_na = round((sum_na / (sum_na + sum_male + sum_female)) * 100, 2))

For more than 74% of trips involving an occasional ticket, the gender of the rider is not known, so we will not consider these statistics.  

For over 99% of members, the gender of the rider is known, and we observe that more trips are made by men (81%) than by women (19%). 
 
## Birth year

We know that birth year data is only available for 2019, so we'll only be looking at this data.

In [ ]:
# How many missing values in birth_year column for 2019 data
cyclistic_ready_for_analysis %>%
  filter(year == 2019) %>%
  summarise(
    sum_na = sum(is.na(birth_year)),
    total_rows = n()) %>%
  mutate(
    percentage_na = round((sum_na / total_rows) * 100, 2))

In [ ]:
# Create a histogram showing the distribution of recorded birth
ggplot(cyclistic_ready_for_analysis %>% 
         filter(!is.na(birth_year)), 
       aes(x = birth_year)) + 
  geom_histogram(binwidth = 1, fill = "steelblue", color = "black")

Few users born after 1995. There's a peak around the 90s and then a gradual decline.

## Trip duration

In [ ]:
# Mean of trip duration in seconds converted in minutes
M_trip_duration_min <- mean(cyclistic_ready_for_analysis$trip_duration) / 60
# Standard deviation in seconds
ET_trip_duration_min <- sd(cyclistic_ready_for_analysis$trip_duration) / 60

# Print mean of trip duration un minutes
print(paste("Mean of trip duration :", round(M_trip_duration_min, 0), "minutes"))

# Print standard deviation in minutes
print(paste("Standard deviation of journey time :", round(ET_trip_duration_min, 0), "minutes"))

In [ ]:
# Mean and standard deviation by type of travel pass (member vs casual)
cyclistic_ready_for_analysis %>%
  group_by(travel_pass) %>%
  summarise(
    M_trip_duration_min_by_pass = mean(trip_duration) / 60,
    ET_trip_duration_min_by_pass = sd(trip_duration) / 60) %>%
  mutate(
    M_trip_duration_min_by_pass = round(M_trip_duration_min_by_pass, 0),
    ET_trip_duration_min_by_pass = round(ET_trip_duration_min_by_pass, 0))

For both types of pass, the standard deviation is much higher than the average trip duration. However, we observe that trips using a temporary pass (casual) are longer (36 minutes on average, standard deviation: 75 minutes) than trips using a member pass (11 minutes on average, standard deviation: 21 minutes).

In [ ]:
# Mean and standard deviation for members, by gender (men vs women)
cyclistic_ready_for_analysis %>%
  filter(travel_pass == "member") %>%
  group_by(gender) %>%
  summarise(
    M_trip_duration_min_by_pass = mean(trip_duration) / 60,
    ET_trip_duration_min_by_pass = sd(trip_duration) / 60) %>%
  mutate(
    M_trip_duration_min_by_pass = round(M_trip_duration_min_by_pass, 0),
    ET_trip_duration_min_by_pass = round(ET_trip_duration_min_by_pass, 0))

If we look at the differences in average trip duration for members between men and women, we realize that there isn't a big difference. However, we notice the standard deviation is lower for women than for men.

## Days of week

We know that there are no missing values in the departure_day_of_week and destination_day_of_week columns.

We checked the statistics were similar for the number of trips per day for departures and arrivals, and then collated the data to see the use by day of the week indifferently from departure or arrival.

In [ ]:
# Add departure_day_of_week and destination_day_of_week in a unique column
cyclistic_long <- cyclistic_ready_for_analysis %>%
  pivot_longer(
    cols = c(departure_day_of_week, destination_day_of_week),
    values_to = "day_of_week")

# Number of trips per day and by travel pass
cyclistic_summary <- cyclistic_long %>%
  group_by(travel_pass, day_of_week) %>%
  summarise(nb_trip_by_day = n(), .groups = "keep") %>%
  
# percentage of trips by day compared to total trips
  group_by(travel_pass) %>%
  mutate(
    total_trips_all_days = sum(nb_trip_by_day),
    percentage_trip_by_day = round((nb_trip_by_day / total_trips_all_days) * 100, 1))

# Print result
cyclistic_summary

In [ ]:
# Print easy read result
ggplot(data = cyclistic_summary %>% filter(!is.na(day_of_week))) +
  geom_bar(stat = "identity", 
           mapping = aes(x = factor(day_of_week, levels = c("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday")), 
                         y = percentage_trip_by_day, fill = travel_pass)) +
  facet_wrap(~travel_pass) +
  theme(axis.text.x = element_text(angle = 45)) +
  labs(title = "Percentage of use by day of week and by travel", 
       x = "Day of week", 
       y = "Percentage of use")


I've chosen to show the percentage of use for each day of the week compared with the rest of the week, rather than the number of journeys, because the number of journeys done by members is much higher than that done by casuals, making the graph difficult to read. This is a trip distribution, not a trip count.  

We can see that casuals use Cyclistic more at weekends, while members use it more during the week.

## Hours

In [ ]:
# Add a column (day_type) to distinct days (week vs week-end) in a temporary dataframe
cyclistic_ready_for_analysis_temporary <- cyclistic_ready_for_analysis %>%
  mutate(
    day_type = ifelse(departure_day_of_week %in% c("Saturday", "Sunday"), "weekend", "week"))

# Merge departure and destination data, duplicates day_type column so that each hour added 
# retains its association with the correct day type
cyclistic_ready_for_analysis_temporary <- cyclistic_ready_for_analysis_temporary %>%
  reframe(
    trip_by_hour = c(departure_hour, destination_hour),
    day_type = rep(day_type, 2)) %>%
  group_by(day_type, trip_by_hour) %>%
  
# Number of trips for each unique combination of day type and time of day
  summarise(nb_trip_by_hour = n(), .groups = 'drop') %>%
  
# Percentage of trips by hour compared to total by type of day
  group_by(day_type) %>%
  mutate(
    total_trips_by_day = sum(nb_trip_by_hour),
    percentage = round((nb_trip_by_hour / total_trips_by_day) * 100, 1))
  
# Print result
cyclistic_ready_for_analysis_temporary

# Peak hours in week
ggplot(cyclistic_ready_for_analysis_temporary %>% 
         filter(day_type == "week"), 
       aes(x = trip_by_hour, y = nb_trip_by_hour)) +
  geom_bar(stat = "identity") +
  labs(title = "Peak hours in week")

# Peak hours in week-end
ggplot(cyclistic_ready_for_analysis_temporary %>% 
         filter(day_type == "weekend"), 
       aes(x = trip_by_hour, y = nb_trip_by_hour)) +
  geom_bar(stat = "identity") +
  labs(title = "Peak hours in week-end")

## Stations

In [ ]:
# Combine departure and destination station names
station_name_favorites <- cyclistic_ready_for_analysis %>%
  reframe(
    station_name_total = c(departure_station_name, destination_station_name)) %>%
  group_by(station_name_total) %>%
  
# Calculate the total number of trips per week: total for each day (departures + arrivals)
  summarise(nb_trip_by_station = n()) %>%
  
# Percentage of each day's trips in relation to total trips
  mutate(
    total_trips_by_station = sum(nb_trip_by_station),
    percentage = round((nb_trip_by_station / total_trips_by_station) * 100, 1)) %>%
  
# Sort results by day with most trips (departures + arrivals)
  arrange(desc(nb_trip_by_station))

# Print result
head(station_name_favorites)

Unfortunately, we can't give you the names of the stations we've never used, as we don't have a complete list of all cyclistic stations.

### Map

If you click on the link below this paragraph, you'll be redirected to a web page hosting two interactive Chicago maps (built with Tableau Public) on which you can see how Cyclistic stations are being used. The larger the dot is, the more frequently the station has been used.  The color of the dots also varies: the darker green the dot is, the less the station is used, the redder the dot is, the more it's used. Fly over the dots and you'll see the name of the station concerned, latitude and longitude, and the number of departures from it (noting that departure and arrival data are similar, we have only dealt with departure data here). There are two maps: one for members, the other for casuals.

Tableau Public :  
<https://public.tableau.com/views/Classeur1_17398130870110/Tableaudebord1?:language=fr-FR&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link>

The map shows the difference in station use between members and casuals. Stations are much less used by the casuals. However, we note different preferences. This probably reflects the purpose of use: utilitarian vs. recreational. Without knowing Chicago and its points of interest, it's difficult to go deeper into this analysis, but I'm sure it will make sense to people familiar with the city.

# Sharing of findings

## Recommendations

* Promote short daily trips (work, study,...):  
- Trips by casuals are longer (36 minutes on average, standard deviation: 75 minutes) than trips by members (11 minutes on average, standard deviation: 21 minutes). Perhaps casuals don't think of themselves as subscribers because the daily effort seems too hight, and short trips should be promoted to them.  

* Favorites stations
- The most frequently used stations are not exactly the same for each type of pass (see interactive map), so it would be interesting to extend the analysis to the places frequented, and imagine targeted communications to initiate the expansion of use to other places and objectives.

* Days and hours of use  

Members seem to use Cyclistic more on weekdays, while casuals seem to prefer to use it at weekends.

On weekdays, activity peaks at the beginning and end of the day (8am then 4pm-5pm), while at weekends usage gradually increases, peaking at 2pm and then gradually decreasing again. 

* Female audience

It would be interesting to obtain more data on the gender of casual users, as we can assume that it would be interesting to target more women:  
- trips completed by “members” are mostly by men (81%) (vs. 19% by women)
- women seem to use Cyclistic with causal pass (but the data aren't robust enough to really support this finding). 
- If we look at the differences in mean trip duration for members between men and women, we realize that there isn't a big difference in usage, but we do realize that the standard deviation is lower for women than for men. Women seem to prefer shorter trips.

### In conclusion:  

* To target casuals (and even more if you're trying to appeal to a female audience), it's a good idea to promote short weekday trips  
* Explore Chicago's topography to understand the preference for using some stations more than others (points of interest, transfers, etc). Target communication on stations preferred by casuals to promote different usage (weekday vs. weekend, short vs. long, utilitarian vs. (but also) recreational).  

## Study limits

* The data only covers the period from January to April 1st, as activity can vary considerably from one season to the next  
* Our main difficulty in this analysis is that the same individual may be represented multiple times in the data, since we only look at trips and do not identify individuals. It is therefore impossible to say how many times the same casual individual occurs  
* Fictitious data as part of an exercise  

## Story of this data

Follow [this link](https://www.canva.com/design/DAGfirR5GW8/1VdObR50kbUGWXuZODCIHg/edit?utm_content=DAGfirR5GW8&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton) to see a slides presentation of this study

# Used tools

## Data origin

Data and exercise from [Data Analytic by Google on Coursera](https://www.coursera.org/professional-certificates/google-data-analytics).

## Analysis and visualization tools

The programming language used is [R](https://www.r-project.org/)

I used as IDE [RStudio Desktop](https://posit.co/download/rstudio-desktop/).  

I used to build vizualisation with latitude and longitude [Tableau Public](https://public.tableau.com/app/discover). 

I used [Canva](https://www.canva.com/) to create the slides presentation

## AI

I used AI as a support tool to gain efficiency (debugging, syntax) and improve my learning by deepening my understanding and exploring new approaches:  

* [DeepSeek](https://www.deepseek.com/)  
* [ChatGPT](https://openai.com/index/chatgpt/)  
* [Perplexity Pro](https://www.perplexity.ai/)  

## Translation

I used [DeepL.com (free version)](https://www.deepl.com/fr/translator) to help me translate my content from French to English.  


Thank you for your interest in my work =^-^=